# 게임 리뷰 데이터 추출하기

## minimap 에서 리뷰를 추출해오는 함수 생성

In [1]:
def get_review(game):
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.common.exceptions import TimeoutException
    from selenium.webdriver.common.keys import Keys
    import pyperclip
    import time

    # Chrome 드라이버 설정
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)

    # 게임 리뷰 데이터를 저장할 reviews 리스트 생성
    reviews = []

    try:
        # 웹페이지 URL
        url = f"https://minimap.net/game/{game}/review?l=kr"

        # 웹페이지 로드
        driver.get(url)

        # 로그인 버튼 찾기 및 클릭
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".C-button .c-button-item.c-color-primary"))  # 실제 로그인 버튼의 CSS 선택자로 변경 필요        
        )
        login_button.click()

        login_button2 = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".pop-sign-up .button-w .login-button-w .login-button"))  # 실제 로그인 버튼의 CSS 선택자로 변경 필요
        )
        login_button2.click()

        # 로그인 팝업 창이 나타날 때까지 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#popupWrapper .popup-wrap-background .popup-dialog-wrap"))  # 실제 팝업 창의 CSS 선택자로 변경 필요
        )

        # 로그인 정보 입력
        id_field = driver.find_element(By.ID, "sign-in-email-input")  # 실제 사용자 이름 필드의 ID로 변경 필요
        pw_field = driver.find_element(By.ID, "sign-in-password-input")  # 실제 비밀번호 필드의 ID로 변경 필요

        pyperclip.copy("lhe9614@gmail.com")
        id_field.send_keys(Keys.CONTROL, 'v')

        pyperclip.copy("!Moon@126419#")
        pw_field.send_keys(Keys.CONTROL, 'v')

        time.sleep(2)

        # 로그인 제출 버튼 클릭
        submit_button = driver.find_element(By.CSS_SELECTOR, ".pop-sign-in .button-w .C-round-button")  # 실제 제출 버튼의 CSS 선택자로 변경 필요
        submit_button.click()

        # 로그인 완료 대기 (예: 특정 요소가 사라질 때까지)
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, "#popupWrapper .popup-wrap-background .popup-dialog-wrap"))
        )

        # 로그인 완료 후 페이지가 완전히 로드될 때까지 대기
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "review-list"))
        )

        # 리뷰 페이지로 다시 이동
        driver.get(url)

        # 리뷰 컨테이너 요소 찾기
        review_container = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "review-list"))
        )

        # 컨테이너 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 스크롤 반복해서 내리기
        ## 최대 반복 수 설정
        max_cnt = 100

        ## 이전 높이 계산
        old_height = driver.execute_script("return document.body.scrollHeight")

        cnt = 0
        while True:
            ## 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            ## 대기
            time.sleep(0.5)
            
            ## 새로운 높이 계산
            new_height = driver.execute_script("return document.body.scrollHeight")
            ## 더 이상 스크롤되지 않으면 종료
            if new_height == old_height:
                break  
            ## 새로운 높이 계산 값을 이전 높이 변수에 저장장 
            old_height = new_height

            ## 카운트 계산
            if cnt == max_cnt:
                break 
            cnt += 1

        # 모든 "post-content" 클래스를 가진 div 요소 찾기
        post_contents = driver.find_elements(By.CLASS_NAME, "post-content")
        
        # 각 "post-content" div의 텍스트 내용 추출
        for post in post_contents:
            review_text = post.text.strip()  # div 내의 모든 텍스트를 가져옴
            reviews.append(review_text)

    except Exception as e:
        print(f"오류 발생, {e}")

    finally:
        # 브라우저 종료
        driver.quit()
    
    return reviews

## game_list 에 해당하는 게임들의 리뷰 추출

In [ ]:
import pandas as pd

# 게임의 리뷰를 가져오기 위한 game_list 라는 변수들의 list 생성
game_list = {
    ('쿠키런킹덤', 'com-devsisters-ck'),
    ('던전앤파이터', 'dungeon-and-fighters'),
    ('메이플스토리', 'maplestory'),
    ('월드오브워크래프트', 'world-of-warcraft'),
    ('블레이드앤소울', 'blade-soul'),
    ('마비노기', 'mabinogi'),
    ('리그오브레전드', 'league-of-legends'),
    ('펜타스톰', 'arena-of-valor'),
    ('펜타스톰', 'com-netmarble-penta-gm'),
    ('도타2', 'dota-2'),
    ('브롤스타즈', 'com-supercell-brawlstars'),
    ('서든어택', 'sudden-attack'),
    ('배틀그라운드', 'playerunknowns-battlegrounds'),
    ('배틀그라운드', 'com-pubg-krmobile'),
    ('오버워치', 'overwatch'),
    ('발로란트', 'valorant'),
    ('클래시오브클랜', 'com-supercell-clashofclans'),
    ('클래시로얄', 'com-supercell-clashroyale-gm'),
    ('스타크래프트', 'starcraft'),
    ('피파온라인', 'fifa-online-4'),
    ('피파23', 'ea-sports-fifa-23'),
    ('캔디크러쉬사가', 'com-king-candycrushsaga'),
    ('마인크래프트', 'minecraft'),
    ('로블록스', 'roblox'),
    ('로블록스', 'com-roblox-client-gm'),
    ('카트라이더', 'kartrider'),
    ('테일즈런너', 'tales-runner'),
    ('크레이지아케이드', 'crazy-arcade'),
    ('모여봐요동물의숲', 'animal-crossing-new-horizons'),
    ('모두의마블', 'com-cjenm-modoomarblekakao-gm'),
    ('포켓몬스터', 'pokemon-scarlet')
}

# 모든 리뷰를 저장할 리스트 생성
all_reviews = []

for game_name, game_id in game_list:
    game_reviews = get_review(game_id)
    
    for review in game_reviews:
        all_reviews.append({
            "Game": game_name,
            "Review": review
        })

# 리뷰 데이터를 데이터프레임으로 변환
game_reviews_df = pd.DataFrame(all_reviews)

# 데이터프레임 출력
print(game_reviews_df)

# CSV 파일로 저장
game_reviews_df.to_csv('game_reviews.csv', index=False)



      Game                                             Review
0     로블록스                     이 게임 하나의 모든 장르에 게임이 있어서 정말 추천함
1     로블록스                                         진짜 할 거 없어요
2     로블록스  게임이라기 보단 게임들이 모여있는 플렛폼에 더 가깝다.\n하지만 게임 저작권 문제해...
3     로블록스  진짜 모든 사람에게 추천드려요!\n왜냐하면 모든 장르의 게임이 다있거든요!\n 컴퓨...
4     로블록스       와!로블록스아시는구나!겁나갓겜입니다\n같은 이미지가 있어서 그렇지 생각보단 ㄱㅊ
..     ...                                                ...
687  테일즈런너  얼라시절 정말 열심히 했었다\n그때쯤 유키가 신캐였던가 그랬지\n난 컨트롤이 그때도...
688  테일즈런너  누구는 추억 보정 잔뜩 들어간 게임이라고 말하지만\n그 당시 이 게임이 가진 매력과...
689  테일즈런너                                             재밌는 테런
690  테일즈런너                                                재밋다
691  테일즈런너               똥망겜이라고 하지만 누구보다 열심히 플레이 했던\n 어린 나...

[692 rows x 2 columns]


## 형태소 분석기를 이용하여 리뷰의 단어 추출

In [7]:
import pandas as pd

# 데이터 불러오기기
data_review = pd.read_csv('./datas/game_reviews.csv')

data_review

,Game,Review
0,로블록스,이 게임 하나의 모든 장르에 게임이 있어서 정말 추천함
1,로블록스,진짜 할 거 없어요
2,로블록스,게임이라기 보단 게임들이 모여있는 플렛폼에 더 가깝다.\n하지만 게임 저작권 문제해...
3,로블록스,진짜 모든 사람에게 추천드려요!\n왜냐하면 모든 장르의 게임이 다있거든요!\n 컴퓨...
4,로블록스,와!로블록스아시는구나!겁나갓겜입니다\n같은 이미지가 있어서 그렇지 생각보단 ㄱㅊ
...,...,...
687,테일즈런너,얼라시절 정말 열심히 했었다\n그때쯤 유키가 신캐였던가 그랬지\n난 컨트롤이 그때도...
688,테일즈런너,누구는 추억 보정 잔뜩 들어간 게임이라고 말하지만\n그 당시 이 게임이 가진 매력과...
689,테일즈런너,재밌는 테런
690,테일즈런너,재밋다


In [2]:
from konlpy.tag import Kkma
from collections import Counter
import pandas as pd
import re

# 데이터 불러오기
data = pd.read_csv('./datas/game_reviews.csv')

# 모든 리뷰를 담을 딕셔너리 생성
game_nouns = {}

# Okt 형태소 분석기 초기화
kkma = Kkma()

# 불용어 지정
stopwords = ["때문", "그때", "정말", "매우", "게임", "로블록스", "모두의마블", "블레이드앤소울", "포켓몬스터", "피파", ]

# 게임별로 리뷰 분석
for game in data['Game'].unique():
    # 해당 게임의 모든 리뷰 추출
    game_reviews = data[data['Game'] == game]['Review']

    # 모든 리뷰에서 명사 추출
    nouns_list = []
    for review in game_reviews:
        if isinstance(review, str):
            review_edit = re.sub("[^0-9a-zA-Z가-힣\\s+]", "", review)
            review_edit_pos = kkma.pos(review_edit)
            sub_list = []
            for word, pos in review_edit_pos:
                if len(word) == 1 or word in stopwords:
                    continue
                if pos.startswith('N'):  # 태그가 N으로 시작하는 모든 명사 추출
                    sub_list.append(word)
            nouns_list.extend(sub_list)
    
    # 게임별 명사 리스트 저장
    game_nouns[game] = nouns_list

    # 그래픽 관련 추가할 명사 : 그래픽, 화면, 화질, 퀄리티, 비디오, 모션
    # 명사+동사 리뷰도 뽑기
    

    print(f"{game} 의 리뷰의 등장하는 명사> ")
    print(set(nouns_list))
    print("")

print(game_nouns)

로블록스 의 리뷰의 등장하는 명사> 
{'제작자', '5000', '민심', '크래프트', '개성', '공간', '하기', '과대', '호환성', '이거', '데이', '버그', '유명', '동접', '피해자', '생각', '어드벤처', '가관', '이미지', '유도', '기본', '성관계', '강요', '대표적', '동안', '전성기', '사실', '최고', '분할', '2017', '요소', '장벽', '현질', '접음', '차지', '선생님', '자기', '의해', '연애', '해본', '버스', '유저', '채팅', '칼날', '옛날', '시스템', '도로', '캐릭터', '정책', '년도', '컴퓨터', '이벤트', '작도', '이드', '단점', '평가', '목록', '세요', '경험', '지금', '유가', '원피스', '프리', '성능', '부모님', '스타', '모임', '등등', '무한', '화재', '감자', '포맷', '록스', '비드', '내용', '초등학생', '조화', '18', '점수', '제목', '비추', '내서', '이매일', '사용', '추천', '메타', '가격', '여담', '400', '고등학생', '화면', '상태', '취향', '티셔츠', '애니메이션', '영상', '목질', '조작', '당장', '누구', '그거', '타도', '오류', '회사', '이건', '월드', '언어', '14', '노트북', '허락', '제대', '라이프', '야한', '개념', '인간', '양날', '38', '이브', '자주', '이것', '진짜', '40', '독차지', '신뢰도', '영어', '천명', '개가', '로그', '정작', '건설', '입양', '시도', '처음', '유료', '자유', '중생', '자동차', '플랫폼', '금지', '상상력', '잔당', '유입', '선택', '문제', '캐릭', '있음', '모바일', '한국어', '망상', '93', '시고', '무료', '최적화', '블록', '세부', '고전', 

In [96]:
game_nouns.keys()

dict_keys(['로블록스', '모두의마블', '블레이드앤소울', '포켓몬스터', '피파', '리그오브레전드', '슈퍼스타', '배틀그라운드', '마인크래프트', '스타크래프트', '모여봐요동물의숲', '발로란트', '브롤스타즈', '이사만루', '월드오브워크래프트', '클래시오브클랜', '카트라이더', '던전앤파이터', '캔디크러쉬사가', '펜타스톰', '도타2', '서든어택', '메이플스토리', '마비노기', '크레이지아케이드', '쿠키런킹덤', '클래시로얄', '오버워치', '마구마구', '테일즈런너'])

In [ ]:
# apriori
from apyori import apriori

# 데이터 불러오기
data = pd.read_csv('./datas/game_reviews.csv')

game_reviews = data[data['Game'] == game]['Review']

result = list(apriori(game_reviews))
data = pd.DataFrame(result)
data

# result = list(apriori(game_nouns_values))
# print(apriori(game_nouns_values))
# data = pd.DataFrame(result)
# data

## 그래픽에 대한 리뷰 추출

## 분위기에 대한 리뷰 추출

# 게임별 리뷰의 워드클라우드

In [109]:
from wordcloud import WordCloud
from collections import Counter
from PIL import Image 
import numpy as np 
import matplotlib.pyplot as plt
import koreanize_matplotlib 

# 한글 폰트 설정 (맥OS 기준, 윈도우의 경우 적절한 폰트 경로로 변경 필요)
font_path = 'C:/Windows/Fonts/HMFMOLD.TTF'

# 각 게임별로 워드클라우드 생성
for game, nouns in game_nouns.items():
    # 단어 빈도수 계산
    word_counts = Counter(nouns)

    # 마스크 이미지 가져오기
    mask_img = np.array(Image.open("./images/console.png"))
    
    # 워드클라우드 생성
    wordcloud = WordCloud(
        font_path=font_path,
        width=512, 
        height=512,
        mask=mask_img,
        background_color='white',
        colormap="twilight"
    ).generate_from_frequencies(word_counts)
    
    # 워드클라우드 그리기
    plt.figure(figsize=(16, 16))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"{game} 리뷰 워드클라우드 분석")
    # plt.show()
    
    # 이미지 저장
    plt.savefig(f"{game}_wordcloud.png")
    plt.close()

print("모든 게임의 워드클라우드가 생성되었습니다.")


모든 게임의 워드클라우드가 생성되었습니다.
